# Ingest Constructors Data

In [0]:
dbutils.widgets.text("env", "dev", "Environment")

env = dbutils.widgets.get("env")

In [0]:
%run ../config $env=$env

In [0]:
%run ../utils

In [0]:
csv_file_path = raw_data_folder_path + "constructors.csv"

df_constructors = (
    spark.read.format("csv")
    .option("header", True)
    .option("inferSchema", True)
    .load(csv_file_path)
)

df_constructors = fillna_str(df_constructors, r"\N")

df_constructors.display() if env == "dev" else None

In [0]:
import pyspark.sql.functions as F

df_transformed = (
    df_constructors
    .withColumnRenamed("constructorId", "constructor_id")
    .withColumnRenamed("constructorRef", "constructor_ref")
    .withColumn("ingestion_date", F.current_timestamp())
    .drop("url")
)

df_transformed.display() if env == "dev" else None

In [0]:
df_transformed.write.format("delta").saveAsTable("bronze_tbl_constructors", mode="overwrite")